# Coleta de dados dos municípios brasileiros

- Fontes: https://cidades.ibge.gov.br/ - IBGE


- Objetivo: Coletar os dados disponíveis no portal do IBGE referente ao último censo (na data de criação deste notebook, trata-se do censo de 2010), dos municípios brasileiros, e os salvar em um .csv, devidamente limpo e formatado.

## Por que Selenium?

O conteúdo do Portal do IBGE é gerado de forma dinâmica, sendo que algumas informações estão "escondidas" e requerem interações do usuário (movimentos do mouse, cliques, etc) para serem disponibilizadas. O Selenium permite simular o comportamento de usuários humanos, expondo as informações relevantes e permitindo, por fim, que se faça a coleta dos dados.

## Importando as bibliotecas e definindo constantes e variáveis iniciais

> Para que o script funcione é necessário estar com o Selenium devidamente instalado, com o *chromedriver*. Para mais informações sobre a instalação do Selenium, confira a documentação [aqui](https://selenium-python.readthedocs.io/installation.html).

In [17]:
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.expected_conditions import (
    presence_of_element_located,
    element_to_be_clickable
)
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

PORTAL = "https://cidades.ibge.gov.br"

# Prevent loading images, as it would only slower the code execution.
option = ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

webdriver = Chrome(options=option)
webdriverwait = WebDriverWait(webdriver, 60)

data = []

## Função para coleta das informações

Quando a página de um muncípio for acessada, é necessário que as informações sejam coletadas. Isto é definido na função a seguir, de modo que:
- "cidade" = nome da cidade.
- "estado" = nome do estado.
- "populacao" = população da cidade no último censo.
- "densidade" = densidade demográfica da cidade, em habitante por km².
- "salario_medio" = salário médio mensal dos trabalhadores da cidade, em salários mínimos.
- "populacao_ocupada" = número de pessoas classificadas como ocupadas.
- "ate_meio_salario" = percentual da população da cidade com renda nominal mensal de até meio salário mínimo.
- "escolarizacao" = percentual da escolarização de crianças entre 6 e 14 anos na cidade.
- "pib" = PIB per capita da cidade.
- "receitas_fontes_externas" = percentual de receitas oriundas de fontes externas.
- "receitas_total" = total de receitas realizadas (dividido por 1000).
- "despesas" = total de despesas empenhadas (dividido por 1000).
- "mortalidade_infantil" = número de óbitos por mil nascidos vivos.
- "internacoes_diarreia" = internações por diarréia, por mil habitantes.
- "area" = area da unidade territorial em km².
- "esgotamento_sanitario" = percentual de esgotamento sanitário adequado.
- "arborizacao" = percentual de arborização de vias públicas.
- "urbanizacao" = percentual de urbanização de vias públicas.

> Já faço, neste momento, uma reformatação inicial nos dados coletados, editando as *strings*, removendo pontos, substituindo as vírgulas das casas decimais por pontos e, de modo geral, deixando as *strings* mais próximas dos devidos valores numéricos correspondentes. A conversão de *string* para números poderia ser feita aqui, mas preferi deixar para a etapa de limpeza e tratamento dos dados, a fim de diminuir a complexidade da seguinte função.

In [18]:
def collect_data(webdriver):
    page_values = webdriver.find_elements_by_css_selector('.indicador__valor')
    city_data = {
        "cidade": webdriver.find_element_by_css_selector('div#local h1').text,
        "estado": webdriver.find_elements_by_css_selector('div#local a')[1].text,
        "populacao": page_values[0].text.split()[0].replace('.', ''),
        "densidade": page_values[1].text.split()[0].replace('.', '').replace(',', '.'),
        "salario_medio": page_values[2].text.split()[0].replace(',', '.'),
        "populacao_ocupada": page_values[3].text.split()[0].replace('.', ''),
        "ate_meio_salario": page_values[5].text.split()[0].replace(',', '.'),
        "escolarizacao": page_values[6].text.split()[0].replace(',', '.'),
        "pib": page_values[9].text.split()[0].replace('.', '').replace(',', '.'),
        "receitas_fontes_externas": page_values[10].text.split()[0].replace(',', '.'),
        "receitas_total": page_values[11].text.split()[0].replace('.', '').replace(',', '.'),
        "despesas": page_values[12].text.split()[0].replace('.', '').replace(',', '.'),
        "mortalidade_infantil": page_values[13].text.split()[0].replace(',', '.'),
        "internacoes_diarreia": page_values[14].text.split()[0].replace(',', '.'),
        "area": page_values[15].text.split()[0].replace('.', '').replace(',', '.'),
        "esgotamento_sanitario": page_values[16].text.split()[0].replace(',', '.'),
        "arborizacao": page_values[17].text.split()[0].replace(',', '.'),
        "urbanizacao": page_values[18].text.split()[0].replace(',', '.')
    }
    return city_data

## Coleta dos dados

- Comportamento esperado do script:
    - Entrar na página inicial do portal.
    - Acessar a lista de estados brasileiros.
    - Para cada estado:
        - Acessar a lista de municípios do estado.
        - Para cada município:
            - Coletar o link da página do município.
    - Para cada link:
        - Acessar a página.
        - Coletar os dados.

In [19]:
# Get to the main page and expose the state menu:

webdriver.get(PORTAL)

webdriverwait.until(
    presence_of_element_located((By.CSS_SELECTOR, '.aside_recolhido'))
)
action_chain = ActionChains(webdriver)
action_chain.move_to_element(
    webdriver.find_element_by_css_selector('.aside_recolhido')
).perform()

webdriverwait.until(
    element_to_be_clickable((By.CSS_SELECTOR, 'div#localidade button'))
).click()

webdriverwait.until(
    element_to_be_clickable((By.CSS_SELECTOR, 'li#menu__municipio'))
).click()

# Loop to go through each state:

webdriverwait.until(
    presence_of_element_located((By.CSS_SELECTOR, 'div#segunda-coluna'))
)
states = webdriver.find_elements_by_css_selector('div#segunda-coluna li')
states = states[1:] # Get rid of the 'Todos' option

links_list = []
for state in states:
    webdriverwait.until(
        element_to_be_clickable((By.CSS_SELECTOR, 'div#segunda-coluna li'))
    )
    state.click()
    
    # Loop through each city and collect link to city page
    webdriverwait.until(
        presence_of_element_located((By.CSS_SELECTOR, 'div.por-estado__selecionar-municipio'))
    )
    citys = webdriver.find_elements_by_css_selector('div.municipios a')
    
    for city in citys:
        link = city.get_attribute('href')
        links_list.append(link)
        
# Loops through each city link and collects data

for index, link in enumerate(links_list, start=1):
    # A high number of accesses, in a small amount of time, may cause service instability.
    # The following sleep statement diminishes the issue, waiting 5 min after every 100 accesses.
    if not index % 100:
        sleep(300)
    
    webdriver.get(link + '/panorama')
    webdriverwait.until(
        presence_of_element_located((By.CSS_SELECTOR, '.indicador__valor'))
    )
    data.append(
        collect_data(webdriver)
    )
    
webdriver.close()

## Salva os dados brutos em um csv

A coleta dos dados leva algumas horas e, portanto, quando terminada, convém salvar os dados em um csv imediatamente, de modo que, se ocorrer algum problema, não será necessário refazer a coleta.

In [20]:
import csv

keys = data[0].keys()
with open('temp_data.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data)

## Verificação e limpeza dos dados com Pandas

É necessário realizar algumas verificações, de modo a garantir a integridade dos dados, assim como algumas conversões nos tipos das variáveis.

In [23]:
import pandas as pd

censo = pd.read_csv('temp_data.csv')

censo

,cidade,estado,populacao,densidade,salario_medio,populacao_ocupada,ate_meio_salario,escolarizacao,pib,receitas_fontes_externas,receitas_total,despesas,mortalidade_infantil,internacoes_diarreia,area,esgotamento_sanitario,arborizacao,urbanizacao
0,Acrelândia,Acre,12538,6.94,2.0,869,45.6,95.1,15984.09,-,25276.53,27229.76,11.72,1.6,1807.953,10.9,11.6,7.3
1,Assis Brasil,Acre,6072,1.22,2.2,439,47.1,85.1,13132.06,-,18177.08,17004.91,8.06,2.3,4974.174,23.1,26.5,0
2,Brasiléia,Acre,21398,5.46,1.8,2396,45,90.2,15663.67,-,53414.58,43280.58,17.54,0.6,3916.505,28.4,72.4,2
3,Bujari,Acre,8471,2.79,1.6,1407,44.6,91.1,16380.21,-,21293.77,16796.61,6.51,-,3034.869,18.9,6.8,9.2
4,Capixaba,Acre,8798,5.17,2.4,538,44.6,92.6,15354.37,-,22177.71,18125.25,16.46,0.6,1701.973,33.2,43.8,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,Luzinópolis,Tocantins,2622,9.38,1.5,270,46.8,99,11110.76,97.6,11336.00,10101.95,-,1,278.603,0.3,57.4,0
5566,Marianópolis do Tocantins,Tocantins,4352,2.08,1.8,562,41.8,98.5,23491.05,89.1,16448.35,14276.76,15.15,0.4,2091.374,12.2,40.1,0
5567,Mateiros,Tocantins,2223,0.23,1.3,497,45.2,95.8,48252.43,93.5,13242.97,11266.67,-,-,9657.943,10.4,33.7,0
5568,Maurilândia do Tocantins,Tocantins,3154,4.27,1.7,253,49,97.9,9699.28,97.2,14788.36,13462.50,23.26,0.3,734.533,15.7,75.2,0


In [24]:
censo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cidade                    5570 non-null   object 
 1   estado                    5570 non-null   object 
 2   populacao                 5570 non-null   object 
 3   densidade                 5570 non-null   object 
 4   salario_medio             5570 non-null   float64
 5   populacao_ocupada         5570 non-null   int64  
 6   ate_meio_salario          5570 non-null   object 
 7   escolarizacao             5570 non-null   object 
 8   pib                       5570 non-null   float64
 9   receitas_fontes_externas  5570 non-null   object 
 10  receitas_total            5570 non-null   object 
 11  despesas                  5570 non-null   object 
 12  mortalidade_infantil      5570 non-null   object 
 13  internacoes_diarreia      5570 non-null   object 
 14  area    

### Conversão de variáveis

Como pode ser visto na saída de *censo.info()*, a maioria das variáveis não está no formato numérico adequado.
O seguinte script faz as devidas conversões.

> Vale notar que NaN é do tipo float. Portanto, variáveis com valores faltantes deverão ser deste tipo.

In [25]:
numeric_variables = [
    'populacao',
    'densidade',
    'salario_medio',
    'populacao_ocupada',
    'ate_meio_salario',
    'escolarizacao',
    'pib',
    'receitas_fontes_externas',
    'receitas_total',
    'despesas',
    'mortalidade_infantil',
    'internacoes_diarreia',
    'area',
    'esgotamento_sanitario',
    'arborizacao',
    'urbanizacao'
]

for variable in numeric_variables:
    censo[variable] = pd.to_numeric(censo[variable], errors='coerce')

In [26]:
censo

,cidade,estado,populacao,densidade,salario_medio,populacao_ocupada,ate_meio_salario,escolarizacao,pib,receitas_fontes_externas,receitas_total,despesas,mortalidade_infantil,internacoes_diarreia,area,esgotamento_sanitario,arborizacao,urbanizacao
0,Acrelândia,Acre,12538.0,6.94,2.0,869,45.6,95.1,15984.09,NaN,25276.53,27229.76,11.72,1.6,1807.953,10.9,11.6,7.3
1,Assis Brasil,Acre,6072.0,1.22,2.2,439,47.1,85.1,13132.06,NaN,18177.08,17004.91,8.06,2.3,4974.174,23.1,26.5,0.0
2,Brasiléia,Acre,21398.0,5.46,1.8,2396,45.0,90.2,15663.67,NaN,53414.58,43280.58,17.54,0.6,3916.505,28.4,72.4,2.0
3,Bujari,Acre,8471.0,2.79,1.6,1407,44.6,91.1,16380.21,NaN,21293.77,16796.61,6.51,NaN,3034.869,18.9,6.8,9.2
4,Capixaba,Acre,8798.0,5.17,2.4,538,44.6,92.6,15354.37,NaN,22177.71,18125.25,16.46,0.6,1701.973,33.2,43.8,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,Luzinópolis,Tocantins,2622.0,9.38,1.5,270,46.8,99.0,11110.76,97.6,11336.00,10101.95,NaN,1.0,278.603,0.3,57.4,0.0
5566,Marianópolis do Tocantins,Tocantins,4352.0,2.08,1.8,562,41.8,98.5,23491.05,89.1,16448.35,14276.76,15.15,0.4,2091.374,12.2,40.1,0.0
5567,Mateiros,Tocantins,2223.0,0.23,1.3,497,45.2,95.8,48252.43,93.5,13242.97,11266.67,NaN,NaN,9657.943,10.4,33.7,0.0
5568,Maurilândia do Tocantins,Tocantins,3154.0,4.27,1.7,253,49.0,97.9,9699.28,97.2,14788.36,13462.50,23.26,0.3,734.533,15.7,75.2,0.0


In [27]:
censo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cidade                    5570 non-null   object 
 1   estado                    5570 non-null   object 
 2   populacao                 5565 non-null   float64
 3   densidade                 5565 non-null   float64
 4   salario_medio             5570 non-null   float64
 5   populacao_ocupada         5570 non-null   int64  
 6   ate_meio_salario          5565 non-null   float64
 7   escolarizacao             5565 non-null   float64
 8   pib                       5570 non-null   float64
 9   receitas_fontes_externas  5001 non-null   float64
 10  receitas_total            5474 non-null   float64
 11  despesas                  5474 non-null   float64
 12  mortalidade_infantil      4151 non-null   float64
 13  internacoes_diarreia      5118 non-null   float64
 14  area    

### Dados Faltantes

No portal do IBGE, nem todas as cidades possuem todos os dados.
É uma boa ideia averiguar as entradas faltantes, garantindo que se tratam de dados realmente faltantes e não de erros na captura ou conversão.

Segundo a saída de *censo.info()*, estão completas (sem valores faltantes) as variáveis:
    - cidade;
    - estado;
    - salario_medio;
    - populacao_ocupada;
    - pib;
    - area.
O restante das variáveis possuem:
    - populacao: 5 valores faltantes;
    - densidade: 5 valores faltantes;
    - ate_meio_salario: 5 valores faltantes;
    - escolarizacao: 5 valores faltantes;
    - receitas_fontes_externas: 569 valores faltantes;
    - receitas_total: 96 valores faltantes;
    - despesas: 96 valores faltantes;
    - mortalidade_infantil: 1419 valores faltantes;
    - internacoes_diarreia: 452 valores faltantes;
    - esgotamento: 5 valores faltantes;
    - arborizacao: 19 valores faltantes;
    - urbanizacao: 5 valores faltantes.

As variáveis contendo apenas 5 valores faltantes podem ser verificadas simplesmente acessando as páginas respectivas no portal e conferindo os valores nela contidos. De fato, conforme pode ser visto abaixo, tratam-se das mesmas 5 cidades.

In [28]:
# List missing data from the 'populacao' variable

missing_populacao = censo[censo['populacao'].isna()]
missing_populacao

,cidade,estado,populacao,densidade,salario_medio,populacao_ocupada,ate_meio_salario,escolarizacao,pib,receitas_fontes_externas,receitas_total,despesas,mortalidade_infantil,internacoes_diarreia,area,esgotamento_sanitario,arborizacao,urbanizacao
1541,Paraíso das Águas,Mato Grosso do Sul,NaN,NaN,3.4,1153,NaN,NaN,115498.52,84.7,45459.89,37325.72,NaN,NaN,5061.433,NaN,NaN,NaN
3111,Mojuí dos Campos,Pará,NaN,NaN,1.6,240,NaN,NaN,9919.95,95.3,39438.71,41276.39,21.83,0.1,4988.236,NaN,NaN,NaN
4075,Pinto Bandeira,Rio Grande do Sul,NaN,NaN,2.5,345,NaN,NaN,16939.10,92.4,14467.07,11108.03,NaN,NaN,104.821,NaN,NaN,NaN
4444,Balneário Rincão,Santa Catarina,NaN,NaN,2.0,1873,NaN,NaN,17168.62,50.3,42067.45,35955.44,15.38,0.2,63.420,NaN,NaN,NaN
4607,Pescaria Brava,Santa Catarina,NaN,NaN,2.0,695,NaN,NaN,13121.66,NaN,21004.81,20922.82,NaN,0.5,106.853,NaN,NaN,NaN


Paraíso das Águas (MS), Mojuí dos Campos (PA), Pinto Bandeira (RS), Balneário Rincão (SC) e Pescaria Brava (SC), segundo o portal do IBGE, são municípios criados após o censo de 2010 e, por esta razão, possuem boa parte de seus dados faltantes.

É inviável conferir todos os dados faltantes das variáveis restantes (mortalidade infantil, por exemplo, possui 1419 dados faltantes). Considere, contudo, que uma mesma função realizou a coleta de todas as páginas das cidades. Observando as 5 cidades verificadas acima, nota-se que a função realizou a coleta de forma adequada, mesmo quando a cidade possuia dados faltantes. Pode-se usar esta informação como evidência da integridade dos dados coletados (pelo menos no que diz respeito aos dados faltantes).

### Dados *outliers*

*Outliers* são dados que se comportam de forma diferente dos demais. 
Convém analisar *outliers* apresentados, a fim de verificar se tratam-se de *outliers* reais ou de erros na leitura ou conversão dos valores.

In [65]:
# List entries that contain an outlier in its variable

import numpy as np
from scipy import stats

for variable in numeric_variables:
    print(variable + ':')
    print(censo[variable][(np.abs(stats.zscore(censo[variable])) >= 3)])

populacao:
Series([], Name: populacao, dtype: float64)
densidade:
Series([], Name: densidade, dtype: float64)
salario_medio:
130     4.3
133     3.5
216     3.5
242     3.9
271     3.9
       ... 
5329    3.5
5341    3.7
5346    4.3
5367    3.9
5503    3.9
Name: salario_medio, Length: 86, dtype: float64
populacao_ocupada:
177      507738
537      849711
677      849045
803     1352143
975      658062
1311     369816
1503     293232
1628    1454749
2510    1030056
3056     438512
3314     705172
3582     301449
3678     314728
4083     784708
4322    2524428
4503     315065
4891     461871
4997     366464
5346    5571893
Name: populacao_ocupada, dtype: int64
ate_meio_salario:
Series([], Name: ate_meio_salario, dtype: float64)
escolarizacao:
Series([], Name: escolarizacao, dtype: float64)
pib:
558     253895.58
839      90330.08
862     292397.08
890     119626.35
925     107838.74
          ...    
5194    344847.17
5253    112903.88
5294     95062.51
5356     91322.46
5415    116199.28

Observa-se, entre os outliers, se algum dos valores apresentados não faz sentido, no contexto da variável em questão.

> No caso deste *dataset*, nenhum *outlier* aparenta ter sido coletado de forma errada.

## Salvando o dataset

Uma vez satisfeito com a integridade dos dados, eles podem ser devidamente salvos, neste caso, na forma de um arquivo csv.

In [66]:
censo.to_csv('censo2010.csv', index=False)